In [1]:
!ipython kernel install --user --name=throne_trader_venv

Installed kernelspec throne_trader_venv in /Users/vicky/Library/Jupyter/kernels/throne_trader_venv


In [2]:
!pip install -qU --upgrade pip
!pip install -qU scikit-learn yfinance numpy pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

### Import required modules

In [3]:
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Union

import numpy as np
import pandas as pd
import yfinance
from sklearn.ensemble import GradientBoostingRegressor

### Set ticker and threshold

In [4]:
TICKER = "GOOGL"
THRESHOLD = 5

In [5]:
def get_historical_data(symbol: str,
                        years: int = 1) -> Union[List[Tuple[str, float]], pd.DataFrame]:
    print(f"Downloading stock data for {symbol} for {years} years.")
    start = (datetime.now() - timedelta(days=years * 365)).strftime("%Y-%m-%d")
    end = datetime.now().strftime("%Y-%m-%d")
    stock_data = yfinance.download(symbol, start=start, end=end)
    return [(date.strftime("%Y-%m-%d"), close_price) for date, close_price in
            zip(stock_data.index, stock_data['Close'])]

In [6]:
stock_data = get_historical_data(symbol=TICKER, years=5)

[*********************100%***********************]  1 of 1 completed


In [7]:
dates, close_prices = zip(*stock_data)
x = np.arange(len(dates)).reshape(-1, 1)
y = np.array(close_prices)

### Train a Gradient Boosting Regressor model

In [8]:
model = GradientBoostingRegressor()
model.fit(x, y)

GradientBoostingRegressor()

### Predict the stock prices for the next two days

In [9]:
next_days = np.array([len(dates), len(dates) + 1]).reshape(-1, 1)
predictions = model.predict(next_days)

### Calculate difference between the predictions and generate signals based on those predictions

In [10]:
diff = predictions[1] - predictions[0]

signals = {"ticker": TICKER}
if diff >= THRESHOLD:
    analysis = signals | {"signal": "buy", "signal_rate": "strong buy", "recommendation": "hold"}
elif -THRESHOLD <= diff <= THRESHOLD:
    analysis = signals | {"signal": "hold", "signal_rate": "neutral", "recommendation": "hold"}
else:
    analysis = signals | {"signal": "sell", "signal_rate": "strong sell", "recommendation": "hold"}

In [11]:
analysis

{'ticker': 'GOOGL',
 'signal': 'hold',
 'signal_rate': 'neutral',
 'recommendation': 'hold'}